**QUESTION 1**

**TESTING THE MODEL**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

article = "My name is Neelu. I like to eat mangoes"
inputs = tokenizer(article, return_tensors="pt")

translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"], max_length=30
)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

'मेरा नाम नीलू है, मुझे आम खाना पसंद है।'

**ACTIVATING CUDA**

In [ ]:
!nvidia-smi

Mon Apr 15 05:30:14 2024       

+---------------------------------------------------------------------------------------+

| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |

|-----------------------------------------+----------------------+----------------------+

| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |

| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |

|                                         |                      |               MIG M. |

|=========================================+======================+======================|

|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |

| N/A   40C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |

|                                         |                      |                  N/A |

+-----------------------------------------+--------------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
model=model.to("cuda")

**EXTRACTING RANDOM 1000 LINES FROM THE FILE**

In [ ]:
import random

# Function to read sentences from a file
def read_sentences_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = file.readlines()
    return sentences

# Define the path to your input text files
en_file_path = "test.en"
hi_file_path = "test.hi"
mr_file_path = "test.mr"

# Define the path for the output files
en_output_file_path = "test.en_1000.txt"
hi_output_file_path = "test.hi_1000.txt"
mr_output_file_path = "test.mr_1000.txt"

# Define the number of random sentences to select
num_random_sentences = 1000

# Read sentences from the English file
en_sentences = read_sentences_from_file(en_file_path)
# Read sentences from the Hindi file
hi_sentences = read_sentences_from_file(hi_file_path)
# Read sentences from the Marathi file
mr_sentences = read_sentences_from_file(mr_file_path)

# Get the total number of sentences
total_sentences = len(en_sentences)

# Generate 1000 random indices
random_indices = random.sample(range(total_sentences), num_random_sentences)

# Initialize lists to store selected sentences
selected_en_sentences = []
selected_hi_sentences = []
selected_mr_sentences = []

# Select sentences corresponding to random indices
for idx in random_indices:
    selected_en_sentences.append(en_sentences[idx])
    selected_hi_sentences.append(hi_sentences[idx])
    selected_mr_sentences.append(mr_sentences[idx])

# Write selected sentences to output files
with open(en_output_file_path, 'w', encoding='utf-8') as en_output_file:
    en_output_file.writelines(selected_en_sentences)

with open(hi_output_file_path, 'w', encoding='utf-8') as hi_output_file:
    hi_output_file.writelines(selected_hi_sentences)

with open(mr_output_file_path, 'w', encoding='utf-8') as mr_output_file:
    mr_output_file.writelines(selected_mr_sentences)

print("Selected 1000 random sentences and saved them to test.en_1000.txt, test.hi_1000.txt, and test.mr_1000.txt.")

Selected 1000 random sentences and saved them to test.en_1000.txt, test.hi_1000.txt, and test.mr_1000.txt.


**ENGLISH TO HINDI**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Define input and output file paths
input_file = "test.en_1000.txt"
output_file = "nllb_eng_to_hindi.txt"

# Define batch size
batch_size = 4

# Read input sentences from file
with open(input_file, "r", encoding="utf-8") as f:
    input_sentences = [line.strip() for line in f.readlines()]

# Translate each sentence and write the translations to the output file
with open(output_file, "w", encoding="utf-8") as f_out:
    for i in range(0, len(input_sentences), batch_size):
        batch_sentences = input_sentences[i:i+batch_size]
        inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"],
            max_length=30,
        )
        translated_sentences = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        for translated_sentence in translated_sentences:
            f_out.write(translated_sentence + "\n")

print("Translations written to", output_file)

Translations written to nllb_eng_to_hindi.txt


**HINDI TO ENGLISH**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Define input and output file paths
input_file = "test.hi_1000.txt"
output_file = "nllb_hin_to_eng.txt"

# Define batch size
batch_size = 4

# Read input sentences from file
with open(input_file, "r", encoding="utf-8") as f:
    input_sentences = [line.strip() for line in f.readlines()]

# Translate each sentence and write the translations to the output file
with open(output_file, "w", encoding="utf-8") as f_out:
    for i in range(0, len(input_sentences), batch_size):
        batch_sentences = input_sentences[i:i+batch_size]
        inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"],
            max_length=30,
        )
        translated_sentences = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        for translated_sentence in translated_sentences:
            f_out.write(translated_sentence + "\n")

print("Translations written to", output_file)

Translations written to nllb_hin_to_eng.txt


**HINDI TO MARATHI**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Define input and output file paths
input_file = "test.hi_1000.txt"
output_file = "nllb_hin_to_mar.txt"

# Define batch size
batch_size = 4

# Read input sentences from file
with open(input_file, "r", encoding="utf-8") as f:
    input_sentences = [line.strip() for line in f.readlines()]

# Translate each sentence and write the translations to the output file
with open(output_file, "w", encoding="utf-8") as f_out:
    for i in range(0, len(input_sentences), batch_size):
        batch_sentences = input_sentences[i:i+batch_size]
        inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["mar_Deva"],
            max_length=30,
        )
        translated_sentences = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        for translated_sentence in translated_sentences:
            f_out.write(translated_sentence + "\n")

print("Translations written to", output_file)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


Translations written to nllb_hin_to_mar.txt


**MARATHI TO HINDI**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Define input and output file paths
input_file = "test.mr_1000.txt"
output_file = "nllb_mar_to_hin.txt"

# Define batch size
batch_size = 4

# Read input sentences from file
with open(input_file, "r", encoding="utf-8") as f:
    input_sentences = [line.strip() for line in f.readlines()]

# Translate each sentence and write the translations to the output file
with open(output_file, "w", encoding="utf-8") as f_out:
    for i in range(0, len(input_sentences), batch_size):
        batch_sentences = input_sentences[i:i+batch_size]
        inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"],
            max_length=30,
        )
        translated_sentences = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
        for translated_sentence in translated_sentences:
            f_out.write(translated_sentence + "\n")

print("Translations written to", output_file)


Translations written to nllb_mar_to_hin.txt


**QUESTION 2:- BLEU AND ROGUE SCORES OF ALL TRANSLATIONS BY NLLB**

In [2]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.1 MB/s eta 0:00:00


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d394f41c2e48df1c951705f7b134e4d14dc37b05748f5db1153bf7cd03755b06
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [17]:
!pip install rouge

In [34]:
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu

def load_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        sentences = [line.strip() for line in f if line.strip()]
    return sentences

def calculate_scores(reference_file, translated_file):
    # Load sentences from files
    reference_sentences = load_sentences(reference_file)
    translated_sentences = load_sentences(translated_file)

    # Initialize ROUGE
    rouge = Rouge()

    # Calculate ROUGE scores
    rouge_scores = rouge.get_scores(translated_sentences, reference_sentences, avg=True)

    # Calculate BLEU score
    bleu_score = corpus_bleu([[ref] for ref in reference_sentences], translated_sentences)

    return rouge_scores, bleu_score

def print_scores(rouge_scores, bleu_score):
    print("ROUGE Scores:")
    for metric, values in rouge_scores.items():
        print(f"{metric}:")
        for key, value in values.items():
            print(f"{key}: {value}")
    print("BLEU Score:", bleu_score)

**FOR ENGLISH TO HINDI**

In [38]:
# File paths for reference and candidate sentences
reference_file = 'test.hi_1000.txt'
translated_file = 'nllb_eng_to_hin.txt'

# Calculate scores
rouge_scores, bleu_score = calculate_scores(reference_file, translated_file)

# Print scores
print_scores(rouge_scores, bleu_score)


ROUGE Scores:
rouge-1:
r: 0.5673244464391796
p: 0.6138173950443074
f: 0.5848161609314676
rouge-2:
r: 0.3379879945430296
p: 0.3671030800977633
f: 0.34880633440256265
rouge-l:
r: 0.5268853560781062
p: 0.5695201785034644
f: 0.5429572079803355
BLEU Score: 0.6258782765551912


**FOR HINDI TO ENGLISH**

In [37]:
# Example usage:
input_file = "test.en_1000.txt"
output_file = "nllb_hin_to_eng.txt"

# Calculate scores
rouge_scores, bleu_score = calculate_scores(input_file, output_file)

# Print scores
print_scores(rouge_scores, bleu_score)

ROUGE Scores:
rouge-1:
r: 0.6109130131344749
p: 0.6265071113633824
f: 0.6136309769878373
rouge-2:
r: 0.39097728857825575
p: 0.39794851846258766
f: 0.39061256117595217
rouge-l:
r: 0.5810031725845388
p: 0.5953442800345884
f: 0.58335953711382
BLEU Score: 0.676214216695444


**FOR HINDI TO MARATHI**

In [41]:
# Example usage:
input_file = "test.mr_1000.txt"
output_file = "nllb_hin_to_mar.txt"

# Calculate scores
rouge_scores, bleu_score = calculate_scores(input_file, output_file)

# Print scores
print_scores(rouge_scores, bleu_score)

ROUGE Scores:
rouge-1:
r: 0.3883062874628678
p: 0.4177619649227626
f: 0.39782587920011836
rouge-2:
r: 0.17215114305334045
p: 0.1841238058209423
f: 0.1757473977623874
rouge-l:
r: 0.36241911174839403
p: 0.3894393365191745
f: 0.37112840951840786
BLEU Score: 0.547942429361091


**FOR MARATHI TO HINDI**

In [42]:
# Example usage:
input_file = "test.hi_1000.txt"
output_file = "nllb_mar_to_hin.txt"

# Calculate scores
rouge_scores, bleu_score = calculate_scores(input_file, output_file)

# Print scores
print_scores(rouge_scores, bleu_score)


ROUGE Scores:
rouge-1:
r: 0.5080351124308534
p: 0.5528065916298678
f: 0.5241000342501483
rouge-2:
r: 0.2785240852566996
p: 0.3022761561799483
f: 0.28692531515869024
rouge-l:
r: 0.47426944035394625
p: 0.5154070180988227
f: 0.489094621494122
BLEU Score: 0.5802169830982862
